In [3]:
import google.generativeai as genai
import google.ai.generativelanguage as glm
import os
from dotenv import load_dotenv

load_dotenv()
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

In [4]:
menu_database = { 
    "아메리카노": {"재고": 10, "가격": 5000}, 
    "카페라떼"  : {"재고": 10, "가격": 6000}, 
    "바닐라라떼": {"재고": 10, "가격": 6000}, 
    "카라멜마끼아또": {"재고": 10, "가격": 6500},
    "돌체라떼": {"재고": 10, "가격": 7500}, 
} 

In [15]:
def is_drink_available(drink_name: str)-> bool: 
    """음료가 주문 가능한지 확인한다.

    Args:
        drink_name: 음료명
    """

    if drink_name in menu_database: 
        if menu_database[drink_name]["재고"] > 0: 
            return True 
    return False 

def get_drink_price(drink_name: str)-> int: 
    """음료의 가격을 가져온다.

    Args:
        drink_name: 음료명
    """
    if drink_name in menu_database: 
        return menu_database[drink_name]["가격"] 
    return None 

def place_order(drink_name: str, address: str)-> str: 
    """음료 주문결과를 반환한다.
    Args:
        drink_name: 음료명
        address: 배송지
    """
    if is_drink_available(drink_name): 
        menu_database[drink_name]["재고"] -= 1 
        return "주문 완료" 
    else: 
        return "재고 부족으로 주문 불가" 

function_repoistory = {     
    "is_drink_available": is_drink_available, 
    "get_drink_price": get_drink_price, 
    "place_order": place_order 
} 

# function_repoistory = [is_drink_available, get_drink_price, place_order]

In [16]:
def correct_response(response): 
    part = response.candidates[0].content.parts[0] 
    if part.function_call: 
        for k, v in part.function_call.args.items(): 
            byte_v = bytes(v, "utf-8").decode("unicode_escape") 
            corrected_v = bytes(byte_v, "latin1").decode("utf-8") 
            part.function_call.args.update({k:  corrected_v}) 

In [17]:
model = genai.GenerativeModel(
    model_name="gemini-1.5-flash", 
    tools=function_repoistory.values()
    # tools = function_repoistory
)

chat_session = model.start_chat(history=[])

In [9]:
function_repoistory

[<function __main__.is_drink_available(drink_name: str) -> bool>,
 <function __main__.get_drink_price(drink_name: str) -> int>,
 <function __main__.place_order(drink_name: str, address: str) -> str>]

In [18]:
function_repoistory.values()

dict_values([<function is_drink_available at 0x000001C524394280>, <function get_drink_price at 0x000001C524575B40>, <function place_order at 0x000001C524575D80>])

In [19]:
queries = []
queries.append("아메리카노 한 잔 주문 가능한가요?")
queries.append("아메리카노 가격이 어떻게 되나요?")
queries.append("강동 청년취업사관학교 5층으로 배송부탁드립니다")             

In [20]:
queries

['아메리카노 한 잔 주문 가능한가요?', '아메리카노 가격이 어떻게 되나요?', '강동 청년취업사관학교 5층으로 배송부탁드립니다']

In [31]:
#response = chat_session.send_message(queries[0]) 

In [32]:
#response.candidates[0].content.parts[0].function_call

In [21]:
def run_chat():
    for query in queries: 
        print(f"\n사용자: {query}")     
        # prompt입력
        response = chat_session.send_message(query) 
        correct_response(response) 
        part = response.candidates[0].content.parts[0] 
    
        if part.function_call: 
            function_call =  part.function_call 
            function_name = function_call.name 
            function_args = {k: v for k, v in function_call.args.items()} 
            function_result = function_repoistory[function_name](**function_args) 
            part = glm.Part(
                    function_response=glm.FunctionResponse(
                        name=function_name, 
                        response={ 
                            "content": function_result, 
                        }, 
                    )
                )
            response = chat_session.send_message(part)
            print(f"직원: {response.candidates[0].content.parts[0].text}")

In [22]:
run_chat()


사용자: 아메리카노 한 잔 주문 가능한가요?
직원: 네, 가능합니다.  


사용자: 아메리카노 가격이 어떻게 되나요?
직원: 아메리카노 가격은 5000원 입니다. 


사용자: 강동 청년취업사관학교 5층으로 배송부탁드립니다
직원: 주문 완료되었습니다.  
강동 청년취업사관학교 5층으로 배송해 드리겠습니다. 



In [19]:
queries = []
queries.append("아메리카노 20 잔 주문 가능한가요?")
queries.append("아메리카노 가격이 어떻게 되나요?")
queries.append("강동 청년취업사관학교 5층으로 배송부탁드립니다")  